In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

# Model Prompting

In [2]:
from intelligence_layer.core import (
    LuminousControlModel,
    CompleteInput,
    InMemoryTracer,
)

tracer = InMemoryTracer()

model = LuminousControlModel()

prompt = model.to_instruct_prompt(
    instruction="Du bist ein Assistent, der Fragen beantwortet.",
    input="Wann wurde Köln gegründet?",
)
complete_input = CompleteInput(prompt=prompt)
response = model.complete(complete_input, tracer=tracer)
response

No CLIENT_URL specified in environment, using default: https://api.aleph-alpha.com.


CompleteOutput(model_version='20240215', completions=[CompletionResult(log_probs=None, completion=' Köln wurde im Jahr 50 vor Christus von den Römern gegründet.', completion_tokens=None, finish_reason='end_of_text', raw_completion=None)], num_tokens_prompt_total=22, num_tokens_generated=14, optimized_prompt=None)

In [3]:
tracer

Open the ]8;id=950199;http://localhost:3000\Trace Viewer]8;;\ to view the trace.

# AtMan for Source Highlighting

In [4]:
instruction = (
    "Sie sind ein Assistent, der Filmkritiken klassifiziert. Sie müssen entscheiden, ob die Person den Film mochte oder nicht."
    + ' Die Ausgabe sollte nur aus den Labels "mögen" oder "nicht mögen" bestehen.'
)
user_input = (
    "Nachricht: Ich habe gestern einen Film geschaut. Es ging um einen Agenten, der eine Mission erfüllen musste. Es war wirklich spannend."
    + " Zunächst musste der Agent eine Basis infiltrieren. Dort gab es einen Bösewicht. Blablabla. Lorem Ipsum dolor."
)
prompt = model.to_instruct_prompt(instruction=instruction, input=user_input)
tracer = InMemoryTracer()

complete_input = CompleteInput(prompt=prompt)
complete_output = model.complete(complete_input, tracer)
complete_output.completions[0].completion

' mögen'

In [5]:
tracer

Open the ]8;id=369450;http://localhost:3000\Trace Viewer]8;;\ to view the trace.

In [6]:
print(prompt.items[0].text)

Sie sind ein Assistent, der Filmkritiken klassifiziert. Sie müssen entscheiden, ob die Person den Film mochte oder nicht. Die Ausgabe sollte nur aus den Labels "mögen" oder "nicht mögen" bestehen.

Nachricht: Ich habe gestern einen Film geschaut. Es ging um einen Agenten, der eine Mission erfüllen musste. Es war wirklich spannend. Zunächst musste der Agent eine Basis infiltrieren. Dort gab es einen Bösewicht. Blablabla. Lorem Ipsum dolor.

### Response:


In [7]:
target = complete_output.completions[0].completion

In [8]:
from intelligence_layer.core import (
    TextHighlight,
    TextHighlightInput,
)
from aleph_alpha_client import PromptGranularity

In [9]:
input = TextHighlightInput(
    rich_prompt=prompt,
    target=target,
    focus_ranges=["input"],
)

highlight_task = TextHighlight(
    model=LuminousControlModel(),
    threshold=0,
    granularity=PromptGranularity.Sentence,
)

tracer = InMemoryTracer()

output = highlight_task.run(input, tracer)
output

TextHighlightOutput(highlights=[ScoredTextHighlight(start=198, end=246, score=0), ScoredTextHighlight(start=247, end=306, score=0), ScoredTextHighlight(start=307, end=332, score=0.055664062), ScoredTextHighlight(start=333, end=383, score=0), ScoredTextHighlight(start=384, end=412, score=0.016601562), ScoredTextHighlight(start=413, end=423, score=0.0009765625), ScoredTextHighlight(start=424, end=442, score=0.0029296875)])

In [10]:
tracer

Open the ]8;id=528705;http://localhost:3000\Trace Viewer]8;;\ to view the trace.

In [11]:
prompt_text = prompt.items[0].text
for highlight in sorted(
    output.highlights, key=lambda highlight: highlight.score, reverse=True
):
    print("-" * 100)
    print(highlight.score)
    print(prompt_text[highlight.start : highlight.end])

----------------------------------------------------------------------------------------------------
0.055664062
Es war wirklich spannend.
----------------------------------------------------------------------------------------------------
0.016601562
Dort gab es einen Bösewicht.
----------------------------------------------------------------------------------------------------
0.0029296875
Lorem Ipsum dolor.
----------------------------------------------------------------------------------------------------
0.0009765625
Blablabla.
----------------------------------------------------------------------------------------------------
0
Nachricht: Ich habe gestern einen Film geschaut.
----------------------------------------------------------------------------------------------------
0
Es ging um einen Agenten, der eine Mission erfüllen musste.
----------------------------------------------------------------------------------------------------
0
Zunächst musste der Agent eine Basis infilt